In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, round
from pyspark.sql.types import DecimalType

In [3]:
### Set up spark session and configurations
spark = SparkSession.builder \
    .appName("NYCTaxiDataAnalysis") \
    .master("local[2]") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate() # Defaults to local mode without master property

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/07 15:17:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
raw_df = spark.read.parquet("raw_data/yellow_tripdata_2024-01.parquet") # Lazily evaluated, not actually submitted at this point

raw_df.printSchema()
raw_df.show(5) # Submits plan for execution

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------

In [14]:
# Calculate total number of passengers per month
total_num_pass = raw_df.select(sum(raw_df.passenger_count).alias("total_passenger_count"),
                               sum(raw_df.total_amount).cast(DecimalType(10, 2)).alias("total_cost"))
total_num_pass.show()

+---------------------+-----------+
|total_passenger_count| total_cost|
+---------------------+-----------+
|              3782748|79456384.28|
+---------------------+-----------+

